In [1]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day15input.txt")

# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:


In [2]:
test_data = ["Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3"]

*Transform the input / test data into:*
- 0-3: a list of relevant coordinates for that sensor/beacon pair
- 4: manhattan distance from this sensor to this beacon, 
- 5-6: upper and lower limits for rows affected by this sensor, using manhattan distance

In [3]:
def parse_data(data):
    returnData = []
    for d in data:
        pair = d.split(":")
        sensor = pair[0].split(",")
        d1 = eval(sensor[0].removeprefix("Sensor at x="))
        d2 = eval(sensor[1].removeprefix(" y="))
        beacon = pair[1].split(",")
        d3 = eval(beacon[0].removeprefix(" closest beacon is at x="))
        d4 = eval(beacon[1].removeprefix(" y="))
        manhattan = abs(d3-d1) + abs(d4-d2)
        upper = d2+manhattan
        lower = d2-manhattan
        returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        #print(d1, d2, d3, d4)
    return returnData

parsed_test_data = parse_data(test_data)
parsed_elf_data = parse_data(elf_data)


So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

                1    1    2    2
        0    5    0    5    0    5
    0 ....S.......................
    1 ......................S.....
    2 ...............S............
    3 ................SB..........
    4 ............................
    5 ............................
    6 ............................
    7 ..........S.......S.........
    8 ............................
    9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....


*This solution could work without this method now, but it was used in part one*

In [4]:
def get_grid_dim(pd):
    min_x = pd[0][0]
    max_x = pd[0][0]
    # min_y = pd[0][1]
    # max_y = pd[0][1]
    for j in pd:
        # print(i)
        # print(j)
        # print(j[0])
        # print(j[1])
        # min_y = min(j[1], j[3], min_y)
        # max_y = max(j[1], j[3], max_y)
        min_x = min(j[0], j[2], min_x)
        max_x = max(j[0], j[2], max_x)

    return (min_x, max_x)


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

                1    1    2    2
        0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
    0 ....S...#####...............
    1 .......#######........S.....
    2 ......#########S............
    3 .....###########SB..........
    4 ....#############...........
    5 ...###############..........
    6 ..#################.........
    7 .#########S#######S#........
    8 ..#################.........
    9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).


*Methods to transform the sensor and beacon data into usable representations in advance, that avoid nested loops within the main tasks*

In [5]:
def get_sensors_or_beacons(parsed_data, min_x, s_or_b):
    coords = {}
    if s_or_b == "S":
        key = 1
    else:
        key = 3
    for p in parsed_data:
    # sensor_x = sensor[0]-min_x
    # sensor_y = sensor[1]  # no need to normalise to 0 index
    # beacon_x = sensor[2]-min_x
    # beacon_y = sensor[3]
    # s0 = s0+1
    # print("Checking sensor ", s0, " of ", s)
        x = p[key-1] - min_x
        if p[key] in coords.keys():
            if key == 1 or (key == 3 and x not in coords[p[key]]):
                coords[p[key]].append(x)
        else:
            coords[p[key]] = [x]
    return coords
        
        
def get_coords_for_all(min_x, max_x, parsed_data):
    coords = {}
    for sensor in parsed_data:
        for row_no in range(sensor[6], sensor[5]+1): #shoudl this be sensor[5]+1?
            x_range = sensor[4] - abs(sensor[1] - row_no) 
            x_left = sensor[0]-min_x - x_range
            if x_left<min_x:
                x_left = min_x
                #print("LEFT, x_left now ", x_left)
            x_right = sensor[0]-min_x + x_range
            if x_right>max_x:
                x_right = max_x
                #print("RIGHT, x_right now ", x_right)
            # if row_no == 0:
            #     print("IN row 0", (x_left, x_right), sensor, x_range, coords.keys())
            if row_no not in coords.keys():
                coords[row_no] = [(x_left, x_right)]
            else: 
                processed = False
                c = -1
                while c<len(coords[row_no])-1 and processed is False:
                    c=c+1
                    if x_left in range(coords[row_no][c][0], coords[row_no][c][1]+1):
                        if x_right not in range(coords[row_no][c][0], coords[row_no][c][1]+1):
                            coords[row_no][c] = (coords[row_no][c][0], x_right)   #extend tuple
                        processed = True #either sorted or we can just discard it as already included
                    else:
                        if x_right in range(coords[row_no][c][0], coords[row_no][c][1]+1):
                            coords[row_no][c] = (x_left, coords[row_no][c][1])   #extend tuple
                            processed = True
                if processed is False and (x_left, x_right) not in coords[row_no]: 
                    coords[row_no].append((x_left, x_right))
            #print("Checking sensor at ", sensor_x, sensor_y, " over range ", x_left, x_right)
    return coords

In [6]:
def process_coords(coords_row, min_x, max_x):

    coords_row.sort()
    (coords_row, all_ruled_out, possible) = remove_overlap(coords_row, min_x, max_x)
    found = not all_ruled_out       
    # print("PROCESS_COORDS coords_row, all_ruled_out", coords_row, all_ruled_out)  
    if found is False:
        return (min_x-1)
    else:
        return possible

def remove_overlap(coords_row, min_x, max_x):
    item = 0
    possibles_set = {-1}
    end_reached = False
    while item < len(coords_row)-1 and end_reached == False:
        #print(item, len(coords_row)-1)
        if coords_row[item][1] >= coords_row[item+1][0]:
            if coords_row[item][1] < coords_row[item+1][1]:
                coords_row[item] = (coords_row[item][0], coords_row[item+1][1])
            
            coords_row.remove(coords_row[item+1])
            
            if item >= len(coords_row)-1:
                end_reached = True
        else: 
            item = item+1
    if len(coords_row) == 1 and coords_row[0] == (min_x, max_x):
        # print("PROCESS COORDS RULED OUT? coords_row, min_x, max_x", coords_row, min_x, max_x)
        all_ruled_out = True
    else:
        # print("PROCESS COORDS FOUND? coords_row, min_x, max_x", coords_row, min_x, max_x)
        all_ruled_out = False
        possibles_set = set(range(min_x, max_x+1))
        for item in coords_row:
            possibles_set = possibles_set - set(range(item[0], item[1]+1))
    possible = min(possibles_set)
    return (coords_row, all_ruled_out, possible)

*We only need to call the set_up_metadata method once for each data/part pair, to set these up in advance. Really I should have done this as a class, with four instances, but I've already hacked this one enough...*


In [7]:
def set_up_metadata(parsed_data, part, is_test):
    if part == 1:
        (min_x, max_x) = get_grid_dim(parsed_data)
        if not is_test:
            min_x = min(min_x, -3000000)
            max_x = max(max_x, 8000000)
        else:
            min_x = min(min_x, -11)
            max_x = max(max_x, 50)
    elif is_test: 
        min_x = 0
        max_x = 20
    else: 
        min_x = 0
        max_x = 4000000
    sensors = get_sensors_or_beacons(parsed_data, min_x, "S")
    beacons = get_sensors_or_beacons(parsed_data, min_x, "B")
    coords = get_coords_for_all(min_x, max_x, parsed_data)
    new_line = []
    for x in range(min_x, max_x+1):    
        new_line.append(".")
    return min_x, max_x, sensors, beacons, coords, new_line
        



*Now that we can set up metadata, the following methods all process data for a row, to detect available/non-available slots in that row*

In [8]:
def setup_row_from_scratch(new_line, sensors_in_row, beacons_in_row, min_x):
    #print(len(new_line))
    # s = len(parsed_data)
    # s0 = 0
    # # print("Empty row set up, checking ", s, " sensors")
    if sensors_in_row is not None:
        for sensor in sensors_in_row:
        # sensor_x = sensor[0]-min_x
        # sensor_y = sensor[1]  # no need to normalise to 0 index
        # beacon_x = sensor[2]-min_x
        # beacon_y = sensor[3]
        # s0 = s0+1
        # print("Checking sensor ", s0, " of ", s)
            new_line[sensor-min_x] = "S"
            # print("sensor added at ", sensor_x, row_num)
    if beacons_in_row is not None:
        for beacon in beacons_in_row:
            new_line[beacon-min_x] =  "B"
            # print("beacon added at ", beacon_x, row_num)
    # print("Finished, returning row")
    return new_line


def check_row_from_scratch(row, row_no, min_x, max_x, coords_in_row, parsed_data):
    if coords_in_row is not None:
        current_left = coords_in_row[0][0]
        current_right = coords_in_row[0][1]
        na = 0
        if current_left>min_x:
            (poss, poss_found) = check_for_availables(row, min_x, current_left, min_x, -1, False)
        else:
            poss = -1
            poss_found = False
        end_reached = False
        i = -1
        while i<(len(coords_in_row)-1) and end_reached == False:
            i = i+1
            #print(coords[i], i, len(coords))
            if coords_in_row[i][0] in range(current_left, current_right+1):  #if x in current coords
                if coords_in_row[i][1] not in range(current_left, current_right+1):    
                    current_right = coords_in_row[i][1]  # extend current coords
            else:  #assume x greater than current, start a new coord, process the last one
                if not poss_found:
                    (poss, poss_found) = check_for_availables(row, current_right+1, coords_in_row[i][0], min_x, poss, poss_found)
                na=na+current_right-current_left
                current_left = coords_in_row[i][0]
                current_right = coords_in_row[i][1]
            if current_right == max_x:
                end_reached = True
                na=na+current_right-current_left
        if not poss_found and (current_right-current_left)<(max_x-min_x):
            (poss, poss_found) = check_for_availables(row, current_right+1, len(row)+min_x, min_x, poss, poss_found)
        if not end_reached:
            na=na+current_right-current_left
    else: 
        na = row.count("S") #+ row.count("B")
        (poss, poss_found) = check_for_availables(row, min_x, max_x, min_x, -1, False)
    return (na, poss)


def check_for_availables(row, start, end, min_x, poss, poss_found):
    temp_slice = row[start-min_x:end-min_x]
    if temp_slice.__contains__("."): #check for available positions 
        poss = temp_slice.index(".")+(start-min_x)
        poss_found = True
    return poss, poss_found


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                    1    1    2    2
        0    5    0    5    0    5
    9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.


*Both part one and part two use the explore_row method - part one just calls it once and part two calls it for all rows it is checking*

In [9]:
def explore_row(row_num, parsed_data, min_x, max_x, sensors, beacons, coords, new_line):
    # print("get grid_dim ", min_x, max_x, " now starting setup row ", row_num)
    nl = new_line.copy()   # having issues here
    if row_num not in sensors.keys():
        sensors_in_row = None
    else: 
        sensors_in_row = sensors[row_num]
    if row_num not in beacons.keys():
        beacons_in_row = None
    else: 
        beacons_in_row = beacons[row_num]
    if row_num not in coords.keys():
        coords_for_row = None
    else:
        coords_for_row = coords[row_num]
    row = setup_row_from_scratch(nl, sensors_in_row, beacons_in_row, min_x)
    # print("Row set up length ", len(row), " starting to check row ", row_num)
    return check_row_from_scratch(row, row_num, min_x, max_x, coords_for_row, parsed_data)

*Some new methods for part two - test method controls whether to execute for part one or two, and the other methods perform part two*

In [10]:
def test(parsed_data, part, is_test, row_num, min_x, max_x, sensors, beacons, coords, new_line):
    if part == 1:
        return explore_row(row_num, parsed_data, min_x, max_x, sensors, beacons, coords, new_line)
    else:
        return look_for_tuning_freq(parsed_data, is_test, min_x, max_x, sensors, beacons, coords, new_line)

def look_for_tuning_freq(parsed_data, is_test, min_x, max_x, sensors, beacons, coords, new_line):
    y = 0
    # if not is_test: 
    #     y = 845967  #hack, as I checked this far by brute force
    found = False
    limit = 20
    if not is_test:
        limit = 4000000
    notavailable3 = max_x
    possible3 = -1
    while y<limit and found is False:
        if y not in coords.keys():
            # print("NO COORDS for ", y)
            possible = min_x
        else:
            coords_for_row = coords[y]
            # print("COORDS ROW", y, coords_for_row)
            possible = process_coords(coords_for_row, min_x, max_x)
        #(notavailable3, possible3) = #explore_row(y, parsed_data, min_x, max_x, sensors, beacons, coords, new_line)
        if y%1000 == 0:
            print("y, possible, min_x, max_x", y, possible, min_x, max_x)
        if possible > min_x-1 and possible-min_x <= max_x:
            found = True
        else: 
            y = y+1
    print("y, limit, found, possible, min_x, max_x", y, limit, found, possible, min_x, max_x)
    return calc_tuning_frequency(possible,y)

def calc_tuning_frequency(x, y):
    return ((x*4000000)+y)

In [11]:
#(parsed_data, part, is_test, row_num, min_x, max_x, sensors, beacons, coords, new_line):
(min_x_t1, max_x_t1, sensors_t1, beacons_t1, coords_t1, new_line_t1) = set_up_metadata(parsed_test_data, 1, True)
print(test(parsed_test_data, 1, True, 10, min_x_t1, max_x_t1, sensors_t1, beacons_t1, coords_t1, new_line_t1))

(26, 0)



In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [14]:
(min_x_e1, max_x_e1, sensors_e1, beacons_e1, coords_e1, new_line_e1) = set_up_metadata(parsed_elf_data, 1, False)
print(test(parsed_elf_data, 1, False, 2000000, min_x_e1, max_x_e1, sensors_e1, beacons_e1, coords_e1, new_line_e1))

(5525990, 0)


# --- Part Two ---

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?



In [12]:
(min_x_t2, max_x_t2, sensors_t2, beacons_t2, coords_t2, new_line_t2) = set_up_metadata(parsed_test_data, 2, True)
# print(min_x_t2, max_x_t2)
# print(coords_t2[0])
#print(sensors_t2)
#print(parsed_test_data)
print(test(parsed_test_data, 2, True, -1, min_x_t2, max_x_t2, sensors_t2, beacons_t2, coords_t2, new_line_t2))
#beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

y, possible, min_x, max_x 0 -1 0 20
y, limit, found, possible, min_x, max_x 11 20 True 14 0 20
56000011


In [13]:
(min_x_e2, max_x_e2, sensors_e2, beacons_e2, coords_e2, new_line_e2) = set_up_metadata(parsed_elf_data, 2, False)
print(test(parsed_elf_data, 2, False, -1, min_x_e2, max_x_e2, sensors_e2, beacons_e2, coords_e2, new_line_e2))

y, possible, min_x, max_x 0 -1 0 4000000
y, possible, min_x, max_x 1000 -1 0 4000000
y, possible, min_x, max_x 2000 -1 0 4000000
y, possible, min_x, max_x 3000 -1 0 4000000
y, possible, min_x, max_x 4000 -1 0 4000000
y, possible, min_x, max_x 5000 -1 0 4000000
y, possible, min_x, max_x 6000 -1 0 4000000
y, possible, min_x, max_x 7000 -1 0 4000000
y, possible, min_x, max_x 8000 -1 0 4000000
y, possible, min_x, max_x 9000 -1 0 4000000
y, possible, min_x, max_x 10000 -1 0 4000000
y, possible, min_x, max_x 11000 -1 0 4000000
y, possible, min_x, max_x 12000 -1 0 4000000
y, possible, min_x, max_x 13000 -1 0 4000000
y, possible, min_x, max_x 14000 -1 0 4000000
y, possible, min_x, max_x 15000 -1 0 4000000
y, possible, min_x, max_x 16000 -1 0 4000000
y, possible, min_x, max_x 17000 -1 0 4000000
y, possible, min_x, max_x 18000 -1 0 4000000
y, possible, min_x, max_x 19000 -1 0 4000000
y, possible, min_x, max_x 20000 -1 0 4000000
y, possible, min_x, max_x 21000 -1 0 4000000
y, possible, min_x, max

In [ ]:
def calc(num_sec_for_1000, mult):
    secs = num_sec_for_1000*(mult/1000)
    print(secs, " secs")
    mins = secs/60
    print(mins, " mins")
    hours = mins/60
    print(hours, " hours")
    days = hours/24
    print(days, " days")
    years = days/365
    print(years, " years")

calc(25, 4000000)

*108000000 is too low*

*got up to 585000*